Voy a tratar de aplicar la deteccion de outliers por lejania o distancia a los datos de este set sobre duraznos.

*Librerias*

In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import scikitplot as skplt 
from sklearn.neighbors import LocalOutlierFactor
np.random.seed(123)

**Ahora leemos el csv, luego de haber corrido el from_exceltoCsv.py**

In [24]:
datos = pd.read_csv("duraznosfinal.csv")
#Eliminamos la primer columna que no nos aporta nada.
datos.drop(datos.columns[0],axis = 1, inplace = True)
datos.head()

,Zona,Finca,Peso,Diámetro Sutural,Diámetro Contrasutural
0,1.0,301,147,64.0,67.0
1,1.0,1502,87,57.0,53.0
2,1.0,301,139,65.0,64.0
3,1.0,301,159,69.0,67.0
4,1.0,1502,128,64.0,62.0


En esta oportunidad probaremos un modelo de LOF. Es un modelo no-supervisado. Mide la desviación local de la 
densidad de una muestra dada con respecto a sus vecinos. Los pasos de computo son:
1. Los *k-vecinos* más cercanos tienen que ser encontrados para cada punto p.
2. Usando estos vecinos *k-nearest-neighbors Nk*, la densidad local para un registro se estima
calculando la densidad de accesibilidad local (LRD)
3. la puntuación **LOF** de un punto p se calcula comparando el **LRD** de un registro
con los **LRD** de sus *k* vecinos

In [25]:
clf  = LocalOutlierFactor(n_neighbors = 20,contamination = 0.1) #20 neighbors es una recomendación que dan todos los libros como numero "magico".

In [26]:
# split into independent variables and target variable
x = datos.drop(['Zona','Finca','Diámetro Sutural','Diámetro Contrasutural'], axis=1)  
# outliers son 9
n_outliers = 9
ground_truth = np.ones(len(x), dtype=int)
ground_truth[-n_outliers:] = -1
# show the shape of x and y  
print("x shape: {}".format(x.shape)) 

x shape: (1186, 1)


In [29]:
y_pred = clf.fit_predict(x)
n_errors = (y_pred != ground_truth).sum()
X_scores = clf.negative_outlier_factor_